In [3]:
# %% [markdown]
# # 1. Ingesta (Capa Bronce)
# Convertimos CSV crudo a formato Delta Lake.

# %%
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
from delta import *
import os

# URL del Master (definida en docker-compose)
master_url = "spark://spark-master:7077"

# Configuración: Añadimos Delta Lake
builder = SparkSession.builder \
    .appName("Lab_SECOP_Bronze") \
    .master(master_url) \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.memory", "1g") 

spark = configure_spark_with_delta_pip(builder).getOrCreate()

# %%
# LECTURA CSV
print("Leyendo CSV crudo...")
df_raw = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("delimiter", ",") \
    .option("inferSchema", "true") \
    .load("../data/SECOP_II_Contratos_Electronicos.csv")


Leyendo CSV crudo...


In [6]:
df_raw.show()

+--------------+-----------+------------+------+------+-----------------------+----------------+-------------------------+---------------------------------------+--------------+----------------------------+-------------------------+-----------+-----------+------------+
|       Entidad|Nit Entidad|Departamento|Ciudad|Estado|Descripcion del Proceso|Tipo de Contrato|Modalidad de Contratacion|Justificacion Modalidad de Contratacion|Fecha de Firma|Fecha de Inicio del Contrato|Fecha de Fin del Contrato|Precio Base|Valor Total|Valor Pagado|
+--------------+-----------+------------+------+------+-----------------------+----------------+-------------------------+---------------------------------------+--------------+----------------------------+-------------------------+-----------+-----------+------------+
|   ALCALDIA 90|       NULL|     BOLIVAR|  NULL|  NULL|                   NULL|            NULL|                     NULL|                                   NULL|    2023-02-28|             

In [9]:
nuevas_columnas= [

    c.strip().replace(" ", "_").replace("(", "").replace(")", "").lower() 
for c in df_raw.columns

]
# Aplicamos los nuevos nombres al DataFrame
df_raw = df_raw.toDF(*nuevas_columnas)
 
print("Nuevas columnas:", df_raw.columns)

Nuevas columnas: ['entidad', 'nit_entidad', 'departamento', 'ciudad', 'estado', 'descripcion_del_proceso', 'tipo_de_contrato', 'modalidad_de_contratacion', 'justificacion_modalidad_de_contratacion', 'fecha_de_firma', 'fecha_de_inicio_del_contrato', 'fecha_de_fin_del_contrato', 'precio_base', 'valor_total', 'valor_pagado']


In [10]:
# %%
# ESCRITURA BRONCE (Delta)
print("Escribiendo en capa Bronce...")
output_path = "../data/lakehouse/bronze/secop"
df_raw.write.format("delta").mode("overwrite").save(output_path)

print(f"Ingesta completada. Registros procesados: {df_raw.count()}")

Escribiendo en capa Bronce...


26/02/02 00:08:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

Ingesta completada. Registros procesados: 1000


In [14]:
delta_secop = "/app/data/lakehouse/bronze/secop"
spark.sql(f"DESCRIBE HISTORY delta.`{delta_secop}`").show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      0|2026-02-02 00:08:...|  NULL|    NULL|    WRITE|{mode -> Overwrit...|NULL|    NULL|     NULL|       NULL|  Serializable|        false|{numFiles -> 1, n...|        NULL|Apache-Spark/3.5....|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+

